# MTG Metagame Analyzer

In [ ]:
# MTG Legacy Metagame Scraper and Analyzer

# %% [markdown]
# Scrape MTGDecks Legacy Metagame (MTGO Events Last 30 Days)
# URL: https://mtgdecks.net/Legacy/metagame:mtgo-events-last-30-days

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

# Tufte-style aesthetics
sns.set(style="white", context="talk", font_scale=1.1)
sns.set_palette("muted")

# Configuration
url = "https://mtgdecks.net/Legacy/metagame:mtgo-events-last-30-days"
top_n = None  # None for all decks, or set to an integer

# Scrape the page with a full browser-like session to avoid 403
# Scrape page using Selenium headless browser to render JS and avoid 403
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium setup
options = Options()
options.headless = True
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Initialize driver (ensure chromedriver is installed and on PATH)
driver = webdriver.Chrome(options=options)
try:
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
finally:
    driver.quit()

# Locate the metagame table (first table on the page)
table = soup.find('table')
if not table:
    raise RuntimeError(f"Metagame table not found at {url}")
rows = table.find_all('tr')[1:]
# Extract deck names and percentages
decks, pcts = [], []
for row in rows:
    name_cell = row.find('td', class_='deck-name')
    pct_cell = row.find('td', class_='deck-percent')
    if name_cell and pct_cell:
        name = name_cell.get_text(strip=True)
        pct_text = pct_cell.get_text(strip=True).strip('%')
        try:
            pct = float(pct_text) / 100.0
        except ValueError:
            continue
        decks.append(name)
        pcts.append(pct)
# Optionally limit to top_n
if top_n:
    decks = decks[:top_n]
    pcts = pcts[:top_n]
if top_n:
    decks = decks[:top_n]
    pcts = pcts[:top_n]
if top_n:
    decks = decks[:top_n]
    pcts = pcts[:top_n]
if top_n:
    decks = decks[:top_n]
    pcts = pcts[:top_n]
if top_n:
    decks = decks[:top_n]
    pcts = pcts[:top_n]

# Build DataFrame
df_meta = pd.DataFrame({"Deck": decks, "Percentage": pcts})
df_meta["Normalized"] = df_meta["Percentage"] / df_meta["Percentage"].sum()
display(df_meta)

# %% [markdown]
# ## Placeholder Winrate Matrix

# %%
winrates = pd.DataFrame(0.5, index=df_meta["Deck"], columns=df_meta["Deck"])
display(winrates)

# %% [markdown]
# ## Projected Winrates and Recommendation

# %%
meta_vec = df_meta["Normalized"].values
projected = winrates.dot(meta_vec)
recommended = projected.idxmax()

# Heatmap of matchups
plt.figure(figsize=(10, 8))
sns.heatmap(winrates, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={"label": "Winrate"})
plt.title("Deck Winrates vs Legacy Metagame", fontsize=16, weight="bold")
plt.xlabel("Opponent Deck")
plt.ylabel("Your Deck")
plt.tight_layout()
plt.show()

# Display results
df_proj = projected.to_frame("Projected Winrate")
print(f"Recommended Deck: {recommended} ({projected[recommended]:.2%})")
display(df_proj)
